# <div class="alert alert-block alert-info">    <b>Package:</b>    scikit learn</div> 

## Import Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn import metrics
import hydroeval as he
import xlsxwriter
import openpyxl
import matplotlib.pyplot as plt
import shap

from sklearn.neighbors import KNeighborsRegressor      # KNN
from sklearn.neural_network import MLPRegressor        # MLP
from sklearn.svm import SVR                            # SVM
from sklearn.tree import DecisionTreeRegressor         # DT
from sklearn.ensemble import RandomForestRegressor     # RF
from sklearn.ensemble import GradientBoostingRegressor # GBM
from sklearn.ensemble import ExtraTreesRegressor       # ET

# LSTM|
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RepeatVector

import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.figure(figsize=(12, 4))
plt.xlabel('Key variables')
plt.show() 

## Load Data

- Output variables are identified with the suffix _eff.

In [ ]:
#Loading data
data = pd.read_excel("C:/Users/sh/Desktop/term 3 ut/Thesis and Paper/R/New folder/MWWTP_impute_data.xlsx")
data_missing = pd.read_excel("C:/Users/sh/Desktop/term 3 ut/Thesis and Paper/R/New folder/MWWTP_raw_data.xlsx")

In [ ]:
# Read column names of data and use it as input(x) and output(y) variables
input_variables = data.columns[2:14]
output_variables = data.columns[0:2]
print(input_variables)
print(output_variables)
train_test_split_point = int(0.8*len(data))

## Create excel files

In [ ]:
import string

excel_name = ['KNN_continuous_data.xlsx',
              'MLP_continuous_data.xlsx',
              'SVM_continuous_data.xlsx',
              'DT_continuous_data.xlsx',
              'RF_continuous_data.xlsx',
              'GBM_continuous_data.xlsx',
              'ET_continuous_data.xlsx',
              'LSTM_continuous_data.xlsx']

for name in excel_name:
    workbook = xlsxwriter.Workbook(name)
    worksheet = workbook.add_worksheet()
    
    for i in range(0,len(output_variables)):
        worksheet.write(''.join(string.ascii_uppercase[i+1] + '1'), output_variables[i])

    worksheet.write('A2', 'RMSE_train')
    worksheet.write('A3', 'RMSE_test')
    worksheet.write('A4', 'R2_train')
    worksheet.write('A5', 'R2_test')
    worksheet.write('A6', 'NSE_train')
    worksheet.write('A7', 'NSE_test')
    workbook.close()

## KNN 
(K nearest neighbor)

In [ ]:
workbook = openpyxl.load_workbook("KNN_continuous_data.xlsx")
w_sheet = workbook["Sheet1"]
df_plot = pd.DataFrame(input_variables,columns=['input'])

column = 2
# output_variables=['Q_eff']
for target in output_variables:
    
    x = data[input_variables].to_numpy()
    y = data[[target]].to_numpy()
        
    # splitting data into train and test
    x_train = x[:train_test_split_point]
    y_train = y[:train_test_split_point]    
    x_test = data[input_variables][train_test_split_point:]
    y_test = data[[target]][train_test_split_point:]
    
    # drop target that is nan in test data
    nan_index = data_missing[target].index[data_missing[target].apply(np.isnan)]
    for nan in nan_index:
        if nan>=train_test_split_point:
            x_test = x_test.drop([nan])
            y_test = y_test.drop([nan])
            
    x_test = x_test.to_numpy()        
    y_test = y_test.to_numpy()    
    
#     from sklearn.decomposition import PCA
#     # Apply PCA to reduce the dimensionality of the data
#     pca = PCA(n_components=10)
#     x_train = pca.fit_transform(x_train)
#     x_test = pca.transform(x_test) 
    
    # Normalization
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    y_train_scaled = scaler.fit_transform(y_train)
    x_test_scaled = scaler.fit_transform(x_test)
    y_test_scaled = scaler.fit_transform(y_test)       

    # Creating the Regression Model
    knn = KNeighborsRegressor()
    
    # Defining the Range of Hyperparameters to Tune
    param_dist = {"n_neighbors": [20,25,30,35,40,45,50],
                  "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
                  "leaf_size": [20,25,30,35,40,45,50,55],
                  "p": [1,1.5,2,2.5,3]}
    
    # Tuning Hyperparameters using Randomized Search Method
    knn_random = RandomizedSearchCV(estimator=knn,
                                    param_distributions=param_dist,
                                    cv=5,
                                    n_iter=10)
    knn_random.fit(x_train_scaled, y_train_scaled.ravel())
    
    print("Target: ", target)
    print("Best Hyperparameters: ", knn_random.best_params_)
    print("Best Score: ", knn_random.best_score_)
    
    # Fitting the Model with Best Hyperparameters
    best_knn = KNeighborsRegressor(n_neighbors=knn_random.best_params_['n_neighbors'],
                                   algorithm=knn_random.best_params_['algorithm'],
                                   leaf_size=knn_random.best_params_['leaf_size'],
                                   p=knn_random.best_params_['p'])
    best_knn.fit(x_train_scaled, y_train_scaled.ravel())

    # Predict target variable in train and test dataset
    predict_train = best_knn.predict(x_train_scaled)
    predict_test = best_knn.predict(x_test_scaled)
    
    # Calculate the accuracy of prediction model  
    predict_train = scaler.inverse_transform(predict_train.reshape(-1, 1))
    predict_test = scaler.inverse_transform(predict_test.reshape(-1, 1))

    nse_train = he.evaluator(he.nse,
                             y_train,
                             predict_train)
    nse_test  = he.evaluator(he.nse,
                             y_test,
                             predict_test)
    
    R2_train = metrics.r2_score(y_train,
                                predict_train)
    R2_test  = metrics.r2_score(y_test,
                                predict_test)
    
    if R2_test < 0:
        corr_matrix_test = np.corrcoef(np.reshape(y_test,
                                                  len(y_test)),
                                       np.reshape(predict_test,
                                                  len(predict_test)))
        R2_test = corr_matrix_test[0,1]**2
        
    if R2_train < 0:
        corr_matrix_train = np.corrcoef(np.reshape(y_train,
                                                   len(y_train)),
                                        np.reshape(predict_train,
                                                   len(predict_train)))
        R2_train = corr_matrix_train[0,1]**2
    
    RMSE_train = np.sqrt(((y_train - predict_train) ** 2).mean())
    RMSE_test = np.sqrt(((y_test - predict_test) ** 2).mean())
    
    # write on excel file
    row = 2
    indices = [round(RMSE_train,2),
               round(RMSE_test,2),
               round(R2_train,2),
               round(R2_test,2),
               round(nse_train[0],2),
               round(nse_test[0],2)]
    
    # iterating through indices list
    for item in indices:
        w_sheet.cell(row=row, column=column).value = item
        row += 1
               
    column += 1
    
    # Plot the predicted values and true values
    plt.figure(figsize=(12, 4))
    plt.margins(x=0,y=0)
    plt.plot(np.vstack((y_train,y_test)), linestyle='-', color='blue', linewidth=0.5)
    plt.plot(np.vstack((predict_train,predict_test)), linestyle='-', color='black', linewidth=0.5)
    plt.fill_between((train_test_split_point, len(data)),
                     min(np.vstack((y_train,y_test,predict_train,predict_test))),
                     max(np.vstack((y_train,y_test,predict_train,predict_test))),
                     facecolor='black', alpha = 0.1)    
    plt.show()

        
    # explain
    explainer = shap.AdditiveExplainer(best_knn.predict, x_train_scaled)
    shap_values = explainer(x_train_scaled)
#     shap.summary_plot(shap_values, features=x_train_scaled, feature_names=data.columns[11:31], plot_type="bar")

    m=[]
    for x in range(0,len(input_variables)):
        n=[]
        for i in range(0,len(shap_values)):
            if shap_values[i][x].values>0:
                n.append(shap_values[i][x].values)
        m.append(np.average(n))
        
    df_plot[target] = m

colors = plt.cm.Greys(np.linspace(0.4, 0.8, 2))
df_plot.plot(x='input', kind='bar', stacked=True, color=colors)
plt.legend(bbox_to_anchor=(1, 1))
plt.title("KNN model")
plt.ylabel('mean(|SHAP value|) (average impact on model output)')
plt.xlabel('input variables')
plt.show() 
    
workbook.save("KNN_continuous_data.xlsx")

## MLP
(Multi layer perceptron)

In [ ]:
workbook = openpyxl.load_workbook("MLP_continuous_data.xlsx")
w_sheet = workbook["Sheet1"]
df_plot = pd.DataFrame(input_variables,columns=['input'])

column = 2
for target in output_variables:
    
    x = data[input_variables].to_numpy()
    y = data[[target]].to_numpy()
        
    # splitting data into train and test
    x_train = x[:train_test_split_point]
    y_train = y[:train_test_split_point]    
    x_test = data[input_variables][train_test_split_point:]
    y_test = data[[target]][train_test_split_point:]

    # drop target that is nan in test data
    nan_index = data_missing[target].index[data_missing[target].apply(np.isnan)]
    for nan in nan_index:
        if nan>=train_test_split_point:
            x_test = x_test.drop([nan])
            y_test = y_test.drop([nan])
            
    x_test = x_test.to_numpy()        
    y_test = y_test.to_numpy()
    
    # Normalization
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    y_train_scaled = scaler.fit_transform(y_train)
    x_test_scaled = scaler.fit_transform(x_test)
    y_test_scaled = scaler.fit_transform(y_test)        

    # Creating the Regression Model
    mlp = MLPRegressor()
    
    # Defining the Range of Hyperparameters to Tune
    param_dist = {"hidden_layer_sizes": [(100,),(150,),(200,),(250,)],
                  "activation": ["identity", "logistic", "tanh", "relu"],
                  "solver": ["lbfgs", "sgd", "adam"],
                  "learning_rate": ["constant", "invscaling", "adaptive"],
                  "learning_rate_init": [0.001]}     
    
    # Tuning Hyperparameters using Randomized Search Method
    mlp_random = RandomizedSearchCV(estimator=mlp,
                                    param_distributions=param_dist,
                                    cv=5,
                                    n_iter=10)
    mlp_random.fit(x_train_scaled, y_train_scaled.ravel())
    
    print("Target: ", target)
    print("Best Hyperparameters: ", mlp_random.best_params_)
    print("Best Score: ", mlp_random.best_score_)
    
    # Fitting the Model with Best Hyperparameters
    best_mlp = MLPRegressor(hidden_layer_sizes=mlp_random.best_params_['hidden_layer_sizes'],
                            activation=mlp_random.best_params_['activation'],
                            solver=mlp_random.best_params_['solver'],
                            learning_rate=mlp_random.best_params_['learning_rate'],
                            learning_rate_init=mlp_random.best_params_['learning_rate_init'])
    best_mlp.fit(x_train_scaled, y_train_scaled.ravel())

    # Predict target variable in train and test dataset
    predict_train = best_mlp.predict(x_train_scaled)
    predict_test = best_mlp.predict(x_test_scaled)
    
    # Calculate the accuracy of prediction model  
    predict_train = scaler.inverse_transform(predict_train.reshape(-1, 1))
    predict_test = scaler.inverse_transform(predict_test.reshape(-1, 1))
    
    nse_train = he.evaluator(he.nse,
                             y_train,
                             predict_train)
    nse_test  = he.evaluator(he.nse,
                             y_test,
                             predict_test)
    
    R2_train = metrics.r2_score(y_train,
                                predict_train)
    R2_test  = metrics.r2_score(y_test,
                                predict_test)
    
    if R2_test < 0:
        corr_matrix_test = np.corrcoef(np.reshape(y_test,
                                                  len(y_test)),
                                       np.reshape(predict_test,
                                                  len(predict_test)))
        R2_test = corr_matrix_test[0,1]**2
        
    if R2_train < 0:
        corr_matrix_train = np.corrcoef(np.reshape(y_train,
                                                   len(y_train)),
                                        np.reshape(predict_train,
                                                   len(predict_train)))
        R2_train = corr_matrix_train[0,1]**2
    
    RMSE_train = np.sqrt(((y_train - predict_train) ** 2).mean())
    RMSE_test = np.sqrt(((y_test - predict_test) ** 2).mean())
    
    # write on excel file
    row = 2
    indices = [round(RMSE_train,2),
               round(RMSE_test,2),
               round(R2_train,2),
               round(R2_test,2),
               round(nse_train[0],2),
               round(nse_test[0],2)]
    
    # iterating through indices list
    for item in indices:
        w_sheet.cell(row=row, column=column).value = item
        row += 1
               
    column += 1
    
    # Plot the predicted values and true values
    plt.figure(figsize=(12, 4))
    plt.margins(x=0,y=0)
    plt.plot(np.vstack((y_train,y_test)), linestyle='-', color='blue', linewidth=0.5)
    plt.plot(np.vstack((predict_train,predict_test)), linestyle='-', color='black', linewidth=0.5)
    plt.fill_between((train_test_split_point, len(data)),
                     min(np.vstack((y_train,y_test,predict_train,predict_test))),
                     max(np.vstack((y_train,y_test,predict_train,predict_test))),
                     facecolor='black', alpha = 0.1)    
    plt.show()

        
    # explain
    explainer = shap.AdditiveExplainer(best_mlp.predict, x_train_scaled)
    shap_values = explainer(x_train_scaled)
#     shap.summary_plot(shap_values, features=x_train_scaled, feature_names=data.columns[11:31], plot_type="bar")

    m=[]
    for x in range(0,len(input_variables)):
        n=[]
        for i in range(0,len(shap_values)):
            if shap_values[i][x].values>0:
                n.append(shap_values[i][x].values)
        m.append(np.average(n))
        
    df_plot[target] = m


colors = plt.cm.Greys(np.linspace(0.4, 0.8, 2))
df_plot.plot(x='input', kind='bar', stacked=True, color=colors)
plt.legend(bbox_to_anchor=(1, 1))
plt.title("MLP model")
plt.ylabel('mean(|SHAP value|) (average impact on model output)')
plt.xlabel('input variables')
plt.show() 
    
workbook.save("MLP_continuous_data.xlsx")

## SVM
(Support-vector machines)

In [3]:
workbook = openpyxl.load_workbook("SVM_continuous_data.xlsx")
w_sheet = workbook["Sheet1"]
df_plot = pd.DataFrame(input_variables,columns=['input'])


column = 2
for target in output_variables:
    
    x = data[input_variables].to_numpy()
    y = data[[target]].to_numpy()
        
    # splitting data into train and test
    x_train = x[:train_test_split_point]
    y_train = y[:train_test_split_point]    
    x_test = data[input_variables][train_test_split_point:]
    y_test = data[[target]][train_test_split_point:]

    # drop target that is nan in test data
    nan_index = data_missing[target].index[data_missing[target].apply(np.isnan)]
    for nan in nan_index:
        if nan>=train_test_split_point:
            x_test = x_test.drop([nan])
            y_test = y_test.drop([nan])
            
    x_test = x_test.to_numpy()        
    y_test = y_test.to_numpy()
    
    # Normalization
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    y_train_scaled = scaler.fit_transform(y_train)
    x_test_scaled = scaler.fit_transform(x_test)
    y_test_scaled = scaler.fit_transform(y_test)     

    # Creating the Regression Model
    svm = SVR()
    
    # Defining the Range of Hyperparameters to Tune
    param_dist = {"tol": [0.000001],
                  "degree": [2,3,4,5],
                  "C": [1,2,3],
                  "gamma": ["scale","auto"]}     
    
    # Tuning Hyperparameters using Randomized Search Method
    svm_random = RandomizedSearchCV(estimator=svm,
                                    param_distributions=param_dist,
                                    cv=5,
                                    n_iter=10)
    svm_random.fit(x_train_scaled, y_train_scaled.ravel())
    
    print("Target: ", target)
    print("Best Hyperparameters: ", svm_random.best_params_)
    print("Best Score: ", svm_random.best_score_)
    
    # Fitting the Model with Best Hyperparameters
    best_svm = SVR(tol=svm_random.best_params_['tol'],
                   C=svm_random.best_params_['C'],
                   degree=svm_random.best_params_['degree'],
                   gamma=svm_random.best_params_['gamma'])
    best_svm.fit(x_train_scaled, y_train_scaled.ravel())

    # Predict target variable in train and test dataset
    predict_train = best_svm.predict(x_train_scaled)
    predict_test = best_svm.predict(x_test_scaled)
    
    # Calculate the accuracy of prediction model  
    predict_train = scaler.inverse_transform(predict_train.reshape(-1, 1))
    predict_test = scaler.inverse_transform(predict_test.reshape(-1, 1))
    
    nse_train = he.evaluator(he.nse,
                             y_train,
                             predict_train)
    nse_test  = he.evaluator(he.nse,
                             y_test,
                             predict_test)
    
    R2_train = metrics.r2_score(y_train,
                                predict_train)
    R2_test  = metrics.r2_score(y_test,
                                predict_test)
    
    if R2_test < 0:
        corr_matrix_test = np.corrcoef(np.reshape(y_test,
                                                  len(y_test)),
                                       np.reshape(predict_test,
                                                  len(predict_test)))
        R2_test = corr_matrix_test[0,1]**2
        
    if R2_train < 0:
        corr_matrix_train = np.corrcoef(np.reshape(y_train,
                                                   len(y_train)),
                                        np.reshape(predict_train,
                                                   len(predict_train)))
        R2_train = corr_matrix_train[0,1]**2
    
    RMSE_train = np.sqrt(((y_train - predict_train) ** 2).mean())
    RMSE_test = np.sqrt(((y_test - predict_test) ** 2).mean())
    
    # write on excel file
    row = 2
    indices = [round(RMSE_train,2),
               round(RMSE_test,2),
               round(R2_train,2),
               round(R2_test,2),
               round(nse_train[0],2),
               round(nse_test[0],2)]
    
    # iterating through indices list
    for item in indices:
        w_sheet.cell(row=row, column=column).value = item
        row += 1
               
    column += 1
    
    # Plot the predicted values and true values
    plt.figure(figsize=(12, 4))
    plt.margins(x=0,y=0)
    plt.plot(np.vstack((y_train,y_test)), linestyle='-', color='blue', linewidth=0.5)
    plt.plot(np.vstack((predict_train,predict_test)), linestyle='-', color='black', linewidth=0.5)
    plt.fill_between((train_test_split_point, len(data)),
                     min(np.vstack((y_train,y_test,predict_train,predict_test))),
                     max(np.vstack((y_train,y_test,predict_train,predict_test))),
                     facecolor='black', alpha = 0.1)    
    plt.show()

        
    # explain
    explainer = shap.AdditiveExplainer(best_svm.predict, x_train_scaled)
    shap_values = explainer(x_train_scaled)
#     shap.summary_plot(shap_values, features=x_train_scaled, feature_names=data.columns[11:31], plot_type="bar")

    m=[]
    for x in range(0,len(input_variables)):
        n=[]
        for i in range(0,len(shap_values)):
            if shap_values[i][x].values>0:
                n.append(shap_values[i][x].values)
        m.append(np.average(n))
        
    df_plot[target] = m

colors = plt.cm.Greys(np.linspace(0.4, 0.8, 2))
df_plot.plot(x='input', kind='bar', stacked=True, color=colors)
plt.legend(bbox_to_anchor=(1, 1))
plt.title("SVM model")
plt.ylabel('mean(|SHAP value|) (average impact on model output)')
plt.xlabel('input variables')
plt.show() 
    
workbook.save("SVM_continuous_data.xlsx")

NameError: name 'openpyxl' is not defined

## DT
(Decision tree)

In [ ]:
workbook = openpyxl.load_workbook("DT_continuous_data.xlsx")
w_sheet = workbook["Sheet1"]
df_plot = pd.DataFrame(input_variables,columns=['input'])

column = 2
for target in output_variables:
    
    x = data[input_variables].to_numpy()
    y = data[[target]].to_numpy()
        
    # splitting data into train and test
    x_train = x[:train_test_split_point]
    y_train = y[:train_test_split_point]    
    x_test = data[input_variables][train_test_split_point:]
    y_test = data[[target]][train_test_split_point:]

    # drop target that is nan in test data
    nan_index = data_missing[target].index[data_missing[target].apply(np.isnan)]
    for nan in nan_index:
        if nan>=train_test_split_point:
            x_test = x_test.drop([nan])
            y_test = y_test.drop([nan])
            
    x_test = x_test.to_numpy()        
    y_test = y_test.to_numpy()
    
    # Normalization
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    y_train_scaled = scaler.fit_transform(y_train)
    x_test_scaled = scaler.fit_transform(x_test)
    y_test_scaled = scaler.fit_transform(y_test)        

    # Creating the Regression Model
    DT = DecisionTreeRegressor()
    
    # Defining the Range of Hyperparameters to Tune
    param_dist = {"max_depth": [3,5,10,15,20],
                  "min_samples_split": [5,10,15,20,25,30,35,40],
                  "min_samples_leaf": [50,100,150,200,250,300]}     
    
    # Tuning Hyperparameters using Randomized Search Method
    DT_random = RandomizedSearchCV(estimator=DT,
                                   param_distributions=param_dist,
                                   cv=5,
                                   n_iter=10)
    DT_random.fit(x_train_scaled, y_train_scaled.ravel())
    
    print("Target: ", target)
    print("Best Hyperparameters: ", DT_random.best_params_)
    print("Best Score: ", DT_random.best_score_)
    
    # Fitting the Model with Best Hyperparameters
    best_DT = DecisionTreeRegressor(max_depth=DT_random.best_params_['max_depth'],
                                    min_samples_split=DT_random.best_params_['min_samples_split'],
                                    min_samples_leaf=DT_random.best_params_['min_samples_leaf'])
    best_DT.fit(x_train_scaled, y_train_scaled.ravel())
#     from sklearn import tree
#     tree.plot_tree(best_DT)

    # Predict target variable in train and test dataset
    predict_train = best_DT.predict(x_train_scaled)
    predict_test = best_DT.predict(x_test_scaled)
    
    # Calculate the accuracy of prediction model  
    predict_train = scaler.inverse_transform(predict_train.reshape(-1, 1))
    predict_test = scaler.inverse_transform(predict_test.reshape(-1, 1))
    
    nse_train = he.evaluator(he.nse,
                             y_train,
                             predict_train)
    nse_test  = he.evaluator(he.nse,
                             y_test,
                             predict_test)
    
    R2_train = metrics.r2_score(y_train,
                                predict_train)
    R2_test  = metrics.r2_score(y_test,
                                predict_test)
    
    if R2_test < 0:
        corr_matrix_test = np.corrcoef(np.reshape(y_test,
                                                  len(y_test)),
                                       np.reshape(predict_test,
                                                  len(predict_test)))
        R2_test = corr_matrix_test[0,1]**2
        
    if R2_train < 0:
        corr_matrix_train = np.corrcoef(np.reshape(y_train,
                                                   len(y_train)),
                                        np.reshape(predict_train,
                                                   len(predict_train)))
        R2_train = corr_matrix_train[0,1]**2
    
    RMSE_train = np.sqrt(((y_train - predict_train) ** 2).mean())
    RMSE_test = np.sqrt(((y_test - predict_test) ** 2).mean())
    
    # write on excel file
    row = 2
    indices = [round(RMSE_train,2),
               round(RMSE_test,2),
               round(R2_train,2),
               round(R2_test,2),
               round(nse_train[0],2),
               round(nse_test[0],2)]
    
    # iterating through indices list
    for item in indices:
        w_sheet.cell(row=row, column=column).value = item
        row += 1
               
    column += 1
    
    # Plot the predicted values and true values
    plt.figure(figsize=(12, 4))
    plt.margins(x=0,y=0)
    plt.plot(np.vstack((y_train,y_test)), linestyle='-', color='blue', linewidth=0.5)
    plt.plot(np.vstack((predict_train,predict_test)), linestyle='-', color='black', linewidth=0.5)
    plt.fill_between((train_test_split_point, len(data)),
                     min(np.vstack((y_train,y_test,predict_train,predict_test))),
                     max(np.vstack((y_train,y_test,predict_train,predict_test))),
                     facecolor='black', alpha = 0.1)    
    plt.show()

        
    # explain
    explainer = shap.AdditiveExplainer(best_DT.predict, x_train_scaled)
    shap_values = explainer(x_train_scaled)
#     shap.summary_plot(shap_values, features=x_train_scaled, feature_names=data.columns[11:31], plot_type="bar")

    m=[]
    for x in range(0,len(input_variables)):
        n=[]
        for i in range(0,len(shap_values)):
            if shap_values[i][x].values>0:
                n.append(shap_values[i][x].values)
        m.append(np.average(n))
        
    df_plot[target] = m

colors = plt.cm.Greys(np.linspace(0.4, 0.8, 2))
df_plot.plot(x='input', kind='bar', stacked=True, color=colors)
plt.legend(bbox_to_anchor=(1, 1))
plt.title("DT model")
plt.ylabel('mean(|SHAP value|) (average impact on model output)')
plt.xlabel('input variables')
plt.show()     

workbook.save("DT_continuous_data.xlsx")

## RF
(Random forest)

In [ ]:
workbook = openpyxl.load_workbook("RF_continuous_data.xlsx")
w_sheet = workbook["Sheet1"]
df_plot = pd.DataFrame(input_variables,columns=['input'])

column = 2
for target in output_variables:
    
    x = data[input_variables].to_numpy()
    y = data[[target]].to_numpy()
        
    # splitting data into train and test
    x_train = x[:train_test_split_point]
    y_train = y[:train_test_split_point]    
    x_test = data[input_variables][train_test_split_point:]
    y_test = data[[target]][train_test_split_point:]

    # drop target that is nan in test data
    nan_index = data_missing[target].index[data_missing[target].apply(np.isnan)]
    for nan in nan_index:
        if nan>=train_test_split_point:
            x_test = x_test.drop([nan])
            y_test = y_test.drop([nan])
            
    x_test = x_test.to_numpy()        
    y_test = y_test.to_numpy()
    
    # Normalization
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    y_train_scaled = scaler.fit_transform(y_train)
    x_test_scaled = scaler.fit_transform(x_test)
    y_test_scaled = scaler.fit_transform(y_test)         

    # Creating the Regression Model
    rf = RandomForestRegressor()
    
    # Defining the Range of Hyperparameters to Tune
    param_dist = {"max_depth": [3,5,10,15,20],
                  "min_samples_split": [2,5,10,15,20,25,30,35,40],
                  "min_samples_leaf": [20,50,100,150,200,250,300]}     
    
    # Tuning Hyperparameters using Randomized Search Method
    rf_random = RandomizedSearchCV(estimator=rf,
                                   param_distributions=param_dist,
                                   cv=5,
                                   n_iter=10)
    rf_random.fit(x_train_scaled, y_train_scaled.ravel())
    
    print("Target: ", target)
    print("Best Hyperparameters: ", rf_random.best_params_)
    print("Best Score: ", rf_random.best_score_)
    
    # Fitting the Model with Best Hyperparameters
    best_rf = RandomForestRegressor(max_depth=rf_random.best_params_['max_depth'],
                                    min_samples_split=rf_random.best_params_['min_samples_split'],
                                    min_samples_leaf=rf_random.best_params_['min_samples_leaf'])
    best_rf.fit(x_train_scaled, y_train_scaled.ravel())

    # Predict target variable in train and test dataset
    predict_train = best_rf.predict(x_train_scaled)
    predict_test = best_rf.predict(x_test_scaled)
    
    # Calculate the accuracy of prediction model  
    predict_train = scaler.inverse_transform(predict_train.reshape(-1, 1))
    predict_test = scaler.inverse_transform(predict_test.reshape(-1, 1))
    
    nse_train = he.evaluator(he.nse,
                             y_train,
                             predict_train)
    nse_test  = he.evaluator(he.nse,
                             y_test,
                             predict_test)
    
    R2_train = metrics.r2_score(y_train,
                                predict_train)
    R2_test  = metrics.r2_score(y_test,
                                predict_test)
    
    if R2_test < 0:
        corr_matrix_test = np.corrcoef(np.reshape(y_test,
                                                  len(y_test)),
                                       np.reshape(predict_test,
                                                  len(predict_test)))
        R2_test = corr_matrix_test[0,1]**2
        
    if R2_train < 0:
        corr_matrix_train = np.corrcoef(np.reshape(y_train,
                                                   len(y_train)),
                                        np.reshape(predict_train,
                                                   len(predict_train)))
        R2_train = corr_matrix_train[0,1]**2
    
    RMSE_train = np.sqrt(((y_train - predict_train) ** 2).mean())
    RMSE_test = np.sqrt(((y_test - predict_test) ** 2).mean())
    
    # write on excel file
    row = 2
    indices = [round(RMSE_train,2),
               round(RMSE_test,2),
               round(R2_train,2),
               round(R2_test,2),
               round(nse_train[0],2),
               round(nse_test[0],2)]
    
    # iterating through indices list
    for item in indices:
        w_sheet.cell(row=row, column=column).value = item
        row += 1
               
    column += 1
    
    # Plot the predicted values and true values
    plt.figure(figsize=(12, 4))
    plt.margins(x=0,y=0)
    plt.plot(np.vstack((y_train,y_test)), linestyle='-', color='blue', linewidth=0.5)
    plt.plot(np.vstack((predict_train,predict_test)), linestyle='-', color='black', linewidth=0.5)
    plt.fill_between((train_test_split_point, len(data)),
                     min(np.vstack((y_train,y_test,predict_train,predict_test))),
                     max(np.vstack((y_train,y_test,predict_train,predict_test))),
                     facecolor='black', alpha = 0.1)    
    plt.show()

        
    # explain
    explainer = shap.AdditiveExplainer(best_rf.predict, x_train_scaled)
    shap_values = explainer(x_train_scaled)
#     shap.summary_plot(shap_values, features=x_train_scaled, feature_names=data.columns[11:31], plot_type="bar")

    m=[]
    for x in range(0,len(input_variables)):
        n=[]
        for i in range(0,len(shap_values)):
            if shap_values[i][x].values>0:
                n.append(shap_values[i][x].values)
        m.append(np.average(n))
        
    df_plot[target] = m

colors = plt.cm.Greys(np.linspace(0.4, 0.8, 2))
df_plot.plot(x='input', kind='bar', stacked=True, color=colors)
plt.legend(bbox_to_anchor=(1, 1))
plt.title("RF model")
plt.ylabel('mean(|SHAP value|) (average impact on model output)')
plt.xlabel('input variables')
plt.show() 
    
workbook.save("RF_continuous_data.xlsx")

## GBM
(Gradient boosting model)

In [ ]:
workbook = openpyxl.load_workbook("GBM_continuous_data.xlsx")
w_sheet = workbook["Sheet1"]
df_plot = pd.DataFrame(input_variables,columns=['input'])


column = 2
for target in output_variables:
    
    x = data[input_variables].to_numpy()
    y = data[[target]].to_numpy()
        
    # splitting data into train and test
    x_train = x[:train_test_split_point]
    y_train = y[:train_test_split_point]    
    x_test = data[input_variables][train_test_split_point:]
    y_test = data[[target]][train_test_split_point:]

    # drop target that is nan in test data
    nan_index = data_missing[target].index[data_missing[target].apply(np.isnan)]
    for nan in nan_index:
        if nan>=train_test_split_point:
            x_test = x_test.drop([nan])
            y_test = y_test.drop([nan])
            
    x_test = x_test.to_numpy()        
    y_test = y_test.to_numpy()
    
    # Normalization
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    y_train_scaled = scaler.fit_transform(y_train)
    x_test_scaled = scaler.fit_transform(x_test)
    y_test_scaled = scaler.fit_transform(y_test)         

    # Creating the Regression Model
    gbm = GradientBoostingRegressor()
    
    # Defining the Range of Hyperparameters to Tune
    param_dist = {"n_estimators": [50,100,150,200,250,300,350],
                  "min_samples_split": [2,5,10,15,20,25,30,35,40],
                  "min_samples_leaf": [20,50,100,150,200,250,300],
                  "learning_rate": [0.01,0.1,0.5],
                  "max_depth": [3,5,10,15,20]}     
    
    # Tuning Hyperparameters using Randomized Search Method
    gbm_random = RandomizedSearchCV(estimator=gbm,
                                   param_distributions=param_dist,
                                   cv=5,
                                   n_iter=10)
    gbm_random.fit(x_train_scaled, y_train_scaled.ravel())
    
    print("Target: ", target)
    print("Best Hyperparameters: ", gbm_random.best_params_)
    print("Best Score: ", gbm_random.best_score_)
    
    # Fitting the Model with Best Hyperparameters
    best_gbm = GradientBoostingRegressor(n_estimators=gbm_random.best_params_['n_estimators'],
                                         min_samples_split=gbm_random.best_params_['min_samples_split'],
                                         min_samples_leaf=gbm_random.best_params_['min_samples_leaf'],
                                         learning_rate=gbm_random.best_params_['learning_rate'],
                                         max_depth=gbm_random.best_params_['max_depth'])
    best_gbm.fit(x_train_scaled, y_train_scaled.ravel())

    # Predict target variable in train and test dataset
    predict_train = best_gbm.predict(x_train_scaled)
    predict_test = best_gbm.predict(x_test_scaled)
    
    # Calculate the accuracy of prediction model  
    predict_train = scaler.inverse_transform(predict_train.reshape(-1, 1))
    predict_test = scaler.inverse_transform(predict_test.reshape(-1, 1))
    
    nse_train = he.evaluator(he.nse,
                             y_train,
                             predict_train)
    nse_test  = he.evaluator(he.nse,
                             y_test,
                             predict_test)
    
    R2_train = metrics.r2_score(y_train,
                                predict_train)
    R2_test  = metrics.r2_score(y_test,
                                predict_test)
    
    if R2_test < 0:
        corr_matrix_test = np.corrcoef(np.reshape(y_test,
                                                  len(y_test)),
                                       np.reshape(predict_test,
                                                  len(predict_test)))
        R2_test = corr_matrix_test[0,1]**2
        
    if R2_train < 0:
        corr_matrix_train = np.corrcoef(np.reshape(y_train,
                                                   len(y_train)),
                                        np.reshape(predict_train,
                                                   len(predict_train)))
        R2_train = corr_matrix_train[0,1]**2
    
    RMSE_train = np.sqrt(((y_train - predict_train) ** 2).mean())
    RMSE_test = np.sqrt(((y_test - predict_test) ** 2).mean())
    
    # write on excel file
    row = 2
    indices = [round(RMSE_train,2),
               round(RMSE_test,2),
               round(R2_train,2),
               round(R2_test,2),
               round(nse_train[0],2),
               round(nse_test[0],2)]
    
    # iterating through indices list
    for item in indices:
        w_sheet.cell(row=row, column=column).value = item
        row += 1
               
    column += 1
    
    # Plot the predicted values and true values
    plt.figure(figsize=(12, 4))
    plt.margins(x=0,y=0)
    plt.plot(np.vstack((y_train,y_test)), linestyle='-', color='blue', linewidth=0.5)
    plt.plot(np.vstack((predict_train,predict_test)), linestyle='-', color='black', linewidth=0.5)
    plt.fill_between((train_test_split_point, len(data)),
                     min(np.vstack((y_train,y_test,predict_train,predict_test))),
                     max(np.vstack((y_train,y_test,predict_train,predict_test))),
                     facecolor='black', alpha = 0.1)    
    plt.show()

        
    # explain
    explainer = shap.AdditiveExplainer(best_gbm.predict, x_train_scaled)
    shap_values = explainer(x_train_scaled)
#     shap.summary_plot(shap_values, features=x_train_scaled, feature_names=data.columns[11:31], plot_type="bar")

    m=[]
    for x in range(0,len(input_variables)):
        n=[]
        for i in range(0,len(shap_values)):
            if shap_values[i][x].values>0:
                n.append(shap_values[i][x].values)
        m.append(np.average(n))
        
    df_plot[target] = m

colors = plt.cm.Greys(np.linspace(0.4, 0.8, 2))
df_plot.plot(x='input', kind='bar', stacked=True, color=colors)
plt.legend(bbox_to_anchor=(1, 1))
plt.title("GBM model")
plt.ylabel('mean(|SHAP value|) (average impact on model output)')
plt.xlabel('input variables')
plt.show() 
    
workbook.save("GBM_continuous_data.xlsx")

## ET
(extra-trees)

In [ ]:
workbook = openpyxl.load_workbook("ET_continuous_data.xlsx")
w_sheet = workbook["Sheet1"]
df_plot = pd.DataFrame(input_variables,columns=['input'])

column = 2
for target in output_variables:
    
    x = data[input_variables].to_numpy()
    y = data[[target]].to_numpy()
        
    # splitting data into train and test
    x_train = x[:train_test_split_point]
    y_train = y[:train_test_split_point]    
    x_test = data[input_variables][train_test_split_point:]
    y_test = data[[target]][train_test_split_point:]

    # drop target that is nan in test data
    nan_index = data_missing[target].index[data_missing[target].apply(np.isnan)]
    for nan in nan_index:
        if nan>=train_test_split_point:
            x_test = x_test.drop([nan])
            y_test = y_test.drop([nan])
            
    x_test = x_test.to_numpy()        
    y_test = y_test.to_numpy()
    
    # Normalization
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    y_train_scaled = scaler.fit_transform(y_train)
    x_test_scaled = scaler.fit_transform(x_test)
    y_test_scaled = scaler.fit_transform(y_test)        

    # Creating the Regression Model
    ET = ExtraTreesRegressor()    
    
    # Defining the Range of Hyperparameters to Tune
    param_dist = {"max_depth": [3,5,10,15,20],
                  "min_samples_split": [2,5,10,15,20,25,30,35,40],
                  "min_samples_leaf": [20,50,100,150,200,250,300],
                  "n_estimators": [20,50,100,150,200,250,300]}     
    
    # Tuning Hyperparameters using Randomized Search Method
    ET_random = RandomizedSearchCV(estimator=ET,
                                   param_distributions=param_dist,
                                   cv=5,
                                   n_iter=10)
    ET_random.fit(x_train_scaled, y_train_scaled.ravel())
    
    print("Target: ", target)
    print("Best Hyperparameters: ", ET_random.best_params_)
    print("Best Score: ", ET_random.best_score_)
    
    # Fitting the Model with Best Hyperparameters
    best_ET = ExtraTreesRegressor(max_depth=ET_random.best_params_['max_depth'],
                                  min_samples_split=ET_random.best_params_['min_samples_split'],
                                  min_samples_leaf=ET_random.best_params_['min_samples_leaf'],
                                  n_estimators=ET_random.best_params_['n_estimators'])
    best_ET.fit(x_train_scaled, y_train_scaled.ravel())

    # Predict target variable in train and test dataset
    predict_train = best_ET.predict(x_train_scaled)
    predict_test = best_ET.predict(x_test_scaled)
        
    # Calculate the accuracy of prediction model  
    predict_train = scaler.inverse_transform(predict_train.reshape(-1, 1))
    predict_test = scaler.inverse_transform(predict_test.reshape(-1, 1))    
    
    nse_train = he.evaluator(he.nse,
                             y_train,
                             predict_train)
    nse_test  = he.evaluator(he.nse,
                             y_test,
                             predict_test)
    
    R2_train = metrics.r2_score(y_train,
                                predict_train)
    R2_test  = metrics.r2_score(y_test,
                                predict_test)
    
    if R2_test < 0:
        corr_matrix_test = np.corrcoef(np.reshape(y_test,
                                                  len(y_test)),
                                       np.reshape(predict_test,
                                                  len(predict_test)))
        R2_test = corr_matrix_test[0,1]**2
        
    if R2_train < 0:
        corr_matrix_train = np.corrcoef(np.reshape(y_train,
                                                   len(y_train)),
                                        np.reshape(predict_train,
                                                   len(predict_train)))
        R2_train = corr_matrix_train[0,1]**2
    
    RMSE_train = np.sqrt(((y_train - predict_train) ** 2).mean())
    RMSE_test = np.sqrt(((y_test - predict_test) ** 2).mean())
    
    # write on excel file
    row = 2
    indices = [round(RMSE_train,2),
               round(RMSE_test,2),
               round(R2_train,2),
               round(R2_test,2),
               round(nse_train[0],2),
               round(nse_test[0],2)]
    
    # iterating through indices list
    for item in indices:
        w_sheet.cell(row=row, column=column).value = item
        row += 1
               
    column += 1
    
    # Plot the predicted values and true values
    plt.figure(figsize=(12, 4))
    plt.margins(x=0,y=0)
    plt.plot(np.vstack((y_train,y_test)), linestyle='-', color='blue', linewidth=0.5)
    plt.plot(np.vstack((predict_train,predict_test)), linestyle='-', color='black', linewidth=0.5)
    plt.fill_between((train_test_split_point, len(data)),
                     min(np.vstack((y_train,y_test,predict_train,predict_test))),
                     max(np.vstack((y_train,y_test,predict_train,predict_test))),
                     facecolor='black', alpha = 0.1)    
    plt.show()
        
        
    # explain
    explainer = shap.AdditiveExplainer(best_ET.predict, x_train_scaled)
    shap_values = explainer(x_train_scaled)
#     shap.summary_plot(shap_values, features=x_train_scaled, feature_names=data.columns[11:31], plot_type="bar")

    m=[]
    for x in range(0,len(input_variables)):
        n=[]
        for i in range(0,len(shap_values)):
            if shap_values[i][x].values>0:
                n.append(shap_values[i][x].values)
        m.append(np.average(n))
        
    df_plot[target] = m

colors = plt.cm.Greys(np.linspace(0.4, 0.8, 2))
df_plot.plot(x='input', kind='bar', stacked=True, color=colors)
plt.legend(bbox_to_anchor=(1, 1))
plt.title("ET model")
plt.ylabel('mean(|SHAP value|) (average impact on model output)')
plt.xlabel('input variables')
plt.show() 
        
        
workbook.save("ET_continuous_data.xlsx")

# <div class="alert alert-block alert-info">    <b>Package:</b>    Keras</div> 

## LSTM
(long-short term memory)

In [ ]:
workbook = openpyxl.load_workbook("LSTM_continuous_data.xlsx")
w_sheet = workbook["Sheet1"]

df_plot = pd.DataFrame(input_variables,columns=['input'])
column = 2
# output_variables=["Q_eff"]
for target in output_variables:
    
    print("Target", target)
    # Split data into training and testing sets
    train_data = data[:train_test_split_point]
    test_data = data[train_test_split_point:]
    
    # Define input and output variables
    x_train_scaled = train_data[input_variables].values
    y_train_scaled = train_data[target].values
    x_test_scaled = test_data[input_variables].values
    y_test_scaled = test_data[target].values
    
    # Normalization
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train_scaled)
    y_train_scaled = y_train_scaled.reshape(-1,1)
    y_train_scaled = scaler.fit_transform(y_train_scaled)
    x_test_scaled = scaler.fit_transform(x_test_scaled)
    y_test_scaled = y_test_scaled.reshape(-1,1)
    y_test_scaled = scaler.fit_transform(y_test_scaled) 
    
    # Reshape input data for LSTM model
    x_train_scaled = np.reshape(x_train_scaled, (x_train_scaled.shape[0], 1, x_train_scaled.shape[1]))
    x_test_scaled = np.reshape(x_test_scaled, (x_test_scaled.shape[0], 1, x_test_scaled.shape[1]))

    # Define LSTM model
    lstm = Sequential()
    lstm.add(LSTM(16, input_shape=(1, len(input_variables)), activation='relu'))
    lstm.add(Dense(2, kernel_initializer='normal', activation='linear'))
    lstm.add(Dense(1, kernel_initializer='normal', activation='linear'))
    lstm.compile(loss='mean_squared_error', optimizer='adam')
    lstm.summary()
    
    # Fit model to training data
    lstm.fit(x_train_scaled, y_train_scaled, epochs=500, batch_size=32, verbose=0)
    
    # Make predictions on testing data
    predict_train = lstm.predict(x_train_scaled)
    predict_test = lstm.predict(x_test_scaled)

    # Calculate the accuracy of prediction model 
    predict_train = scaler.inverse_transform(predict_train.reshape(-1, 1))
    predict_test = scaler.inverse_transform(predict_test.reshape(-1, 1))
    y_train_scaled = scaler.inverse_transform(y_train_scaled.reshape(-1, 1))
    y_test_scaled = scaler.inverse_transform(y_test_scaled.reshape(-1, 1))
    
    nse_train = he.evaluator(he.nse,
                             y_train_scaled,
                             predict_train)
    nse_test  = he.evaluator(he.nse,
                             y_test_scaled,
                             predict_test)
    
    R2_train = metrics.r2_score(y_train_scaled,
                                predict_train)
    R2_test  = metrics.r2_score(y_test_scaled,
                                predict_test)
    
    if R2_test <= 0:
        corr_matrix_test = np.corrcoef(np.reshape(y_test_scaled,
                                                  len(y_test_scaled)),
                                       np.reshape(predict_test,
                                                  len(predict_test)))
        R2_test = corr_matrix_test[0,1]**2
        
    if R2_train <= 0:
        corr_matrix_train = np.corrcoef(np.reshape(y_train_scaled,
                                                   len(y_train_scaled)),
                                        np.reshape(predict_train,
                                                   len(predict_train)))
        R2_train = corr_matrix_train[0,1]**2
    
    RMSE_train = np.sqrt(((y_train_scaled - predict_train) ** 2).mean())
    RMSE_test = np.sqrt(((y_test_scaled - predict_test) ** 2).mean())
    
    # write on excel file
    row = 2
    indices = [round(RMSE_train,2),
               round(RMSE_test,2),
               round(R2_train,2),
               round(R2_test,2),
               round(nse_train[0],2),
               round(nse_test[0],2)]
    print(R2_test)
    
    # iterating through indices list
    for item in indices:
        w_sheet.cell(row=row, column=column).value = item
        row += 1
               
    column += 1 
    
    # Plot the predicted values and true values
    plt.figure(figsize=(12, 4))
    plt.margins(x=0,y=0)
    plt.plot(np.concatenate((y_train_scaled, y_test_scaled), axis=0), linestyle='-', color='blue', linewidth=0.5)
    plt.plot(np.vstack((predict_train,predict_test)), linestyle='-', color='black', linewidth=0.5)   
    plt.fill_between((train_test_split_point, len(data)),
                     min(np.vstack((y_train_scaled,y_test_scaled,predict_train,predict_test))),
                     max(np.vstack((y_train_scaled,y_test_scaled,predict_train,predict_test))),
                     facecolor='black', alpha = 0.1)    
    plt.show()
        
    # explain
    explainer = shap.GradientExplainer(lstm, x_train_scaled)
    shap_values = explainer.shap_values(x_train_scaled)
#     shap.summary_plot(shap_values[0][:, 0, :], feature_names=data.columns[11:31], plot_size=0.2, plot_type="bar")

    m=[]
    for x in range(0,len(input_variables)):
        n=[]
        for i in range(0,len(shap_values[0][:, 0, :])):
            if shap_values[0][:, 0, :][i][x]>0:
                n.append(shap_values[0][:, 0, :][i][x])
        m.append(np.average(n))
        
    df_plot[target] = m

colors = plt.cm.Greys(np.linspace(0.4, 0.8, 2))
df_plot.plot(x='input', kind='bar', stacked=True, color=colors)
plt.legend(bbox_to_anchor=(1, 1))
plt.title("LSTM model")
plt.ylabel('mean(|SHAP value|) (average impact on model output)')
plt.xlabel('input variables')
plt.show() 
    
workbook.save("LSTM_continuous_data.xlsx")
